# PRV Assignment 1

- Pablo Garcia Fernández

Initial configuration

In [57]:
# Initial configuration
import numpy as np
from math import cos,sin

#######################################
## PREDIFINE CONSTATNS OF THE EXERCISE

ROLL=2.5; PITCH=5; YAW=60 
TRANSLATION = np.array([0, 0, 0.25])
######################################

Auxiliar functions

In [58]:
def enu_to_ned_ned_to_enu():
    transformation_matrix = np.array([
        [0., 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, -1, 0],
		[0, 0, 0, 1]
    ])
    return transformation_matrix


def _3D_rotation_matrix(roll, pitch, yaw, order='ypr', inverso=False):
	# Return the rotation matrix in 3D coordinate systems
	yaw = np.array([
		[cos(yaw), -sin(yaw), 0, 0],
		[sin(yaw), cos(yaw), 0, 0],
		[0, 0, 1, 0],
		[0, 0, 0, 1]
	])
	pitch = np.array([
		[cos(pitch), 0, sin(pitch), 0],
		[0, 1, 0, 0],
		[-sin(pitch), 0, cos(pitch), 0],
		[0, 0, 0, 1]
	])
	roll = np.array([
		[1, 0, 0, 0],
		[0, cos(roll), -sin(roll), 0],
		[0, sin(roll), cos(roll), 0],
		[0, 0, 0, 1]
	]) 

	rotation_matrix = None
	
	if order=='ypr':
		if inverso:
			rotation_matrix = yaw.T @ pitch.T @ roll.T
		else:
			rotation_matrix = roll @ pitch @ yaw
	elif order == 'rpy':
		if inverso:
			rotation_matrix = roll.T @ pitch.T @ yaw.T
		else:
			rotation_matrix = yaw @ pitch @ roll
	else:
		print("Order not supported")

	return rotation_matrix

### World CS -> INS CS

In [59]:
def wcs_to_ins():
    # 1) ENU to NED
    t1 = enu_to_ned_ned_to_enu()

    # 2) yaw, pitch, roll
    t2 = _3D_rotation_matrix(ROLL, PITCH, YAW)

    return t2 @ t1

### INS CS -> gimbal CS

In [60]:
def ins_to_gimbal(traslation_vector):
    # 1) NED to ENU
    transformation_matrix = enu_to_ned_ned_to_enu()
    
    # 2) Traslacion
    transformation_matrix[:-1,-1] = -traslation_vector

    return transformation_matrix


### Gimbal -> camera

In [61]:
# Si camara Nadiral orientation -> rotacion 180 en eje e
def gimbal_to_camera(omega=180):
    rotation_mat = np.array([
		[1, 0, 0, 0],
		[0, cos(omega), -sin(omega), 0],
		[0, sin(omega), cos(omega), 0],
		[0, 0, 0, 1]
	]) 

    return rotation_mat

### Composition of all transformations

In [62]:
t1 = wcs_to_ins()
print("wcs to ins:\n", t1)

t2 = ins_to_gimbal(TRANSLATION)
print("ins to gimbal:\n", t2)

t3 = gimbal_to_camera()
print("gimbal to camera:\n",t3)

# TOTAL
t = t3 @ t2 @ t1
print("full matrix:", t)

### Ejemplo de proyeccion
punto = np.array([3, 4, 0, 1])
proyeccion = t @ punto
print("Proyeccion del punto", punto, "a ", proyeccion)

wcs to ins:
 [[ 0.08646325 -0.27016355  0.95892427  0.        ]
 [ 0.58809197  0.79077686  0.16976392  0.        ]
 [-0.80415915  0.54925733  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
ins to gimbal:
 [[ 0.    1.    0.   -0.  ]
 [ 1.    0.    0.   -0.  ]
 [ 0.    0.   -1.   -0.25]
 [ 0.    0.    0.    1.  ]]
gimbal to camera:
 [[ 1.          0.          0.          0.        ]
 [ 0.         -0.59846007  0.80115264  0.        ]
 [ 0.         -0.80115264 -0.59846007  0.        ]
 [ 0.          0.          0.          1.        ]]
full matrix: [[ 0.58809197  0.79077686  0.16976392  0.        ]
 [ 0.59250942 -0.27835686 -0.75594315 -0.20028816]
 [-0.5505274   0.54515082 -0.63224218  0.14961502]
 [ 0.          0.          0.          1.        ]]
Proyeccion del punto [3 4 0 1] a  [4.92738336 0.46381266 0.67863609 1.        ]
